In [17]:
#Modified code from Web Data: Amazon reviews
import gzip
import pandas as pd

def parse(filename):
  f = gzip.open(filename, 'r')
  entry = {}
  for l in f:
    l = l.decode('utf8').strip()
    colonPos = l.find(':')
    if colonPos == -1:
      yield entry
      entry = {}
      continue
    eName = l[:colonPos]
    rest = l[colonPos+2:]
    entry[eName] = rest
  yield entry

#create list with dictionaries, from which we will create dataframe
data = []
for e in parse("data\Cell_Phones_&_Accessories.txt.gz"):
  data.append(e)



In [54]:
#read the data as pandas dataframe
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78931 entries, 0 to 78930
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   product/productId   78930 non-null  object
 1   product/title       78930 non-null  object
 2   product/price       78930 non-null  object
 3   review/userId       78930 non-null  object
 4   review/profileName  78930 non-null  object
 5   review/helpfulness  78930 non-null  object
 6   review/score        78930 non-null  object
 7   review/time         78930 non-null  object
 8   review/summary      78930 non-null  object
 9   review/text         78930 non-null  object
dtypes: object(10)
memory usage: 6.0+ MB


In [55]:
#rename columns
df.columns = ['productID','product','price','userID','profile','helpfulness','score','date','summary','text']
#Convert column types
df = df.replace({'unknown': None})
df[['price','score','date']] = df[['price','score','date']].apply(pd.to_numeric)
df['date'] = pd.to_datetime(df['date'], unit='s')
df[['productID','product','userID','profile','helpfulness','summary','text']] = \
    df[['productID','product','userID','profile','helpfulness','summary','text']].astype("string")


In [57]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78931 entries, 0 to 78930
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   productID    78930 non-null  string        
 1   product      78930 non-null  string        
 2   price        35643 non-null  float64       
 3   userID       76654 non-null  string        
 4   profile      76654 non-null  string        
 5   helpfulness  78930 non-null  string        
 6   score        78930 non-null  float64       
 7   date         78930 non-null  datetime64[ns]
 8   summary      78930 non-null  string        
 9   text         78930 non-null  string        
dtypes: datetime64[ns](1), float64(2), string(7)
memory usage: 6.0 MB
